In [1]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
pd.set_option('display.max_rows', 100)

In [8]:
survey_result_path = '../survey'
survey_result_file_lst = [f for f in listdir(survey_result_path) if isfile(join(survey_result_path, f)) and f.endswith(".csv")]

file_of_interest = survey_result_file_lst[0]

In [53]:
def parse_survey_results(file, verbose = 0):
    with open(join(survey_result_path, file)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        question_index_lst = []
        
        input_dict3_lst = []
        actual_score_lst = []
        gpt2_tuned_score_lst = []
        xlnet_score_lst = []
        lstm_attention_score_lst = []
        gpt2_untuned_bert_score_lst = []
        gpt2_untuned_score_lst = []
        for row in csv_reader:
            if line_count == 0:
                for (i,item) in enumerate(row):
                    if '_' in item:
                        question_index_lst.append(i)
            elif line_count == 1:
                for (i,item) in enumerate(row):
                    if i in question_index_lst:
                        input_dict3_lst.append(int(item.split('. ')[0]))
            elif line_count > 2:
                j = 0
                for (i,item) in enumerate(row):
                    if i in question_index_lst:
                        if len(item) == 0:
                            score_num = ''
                        else:
                            if item == '1':
                                score_num = 1
                            elif item == '2':
                                score_num = 2
                            elif item == '6':
                                score_num = 3
                            elif item == '7':
                                score_num = 4
                            elif item == '8':
                                score_num = 5
                                
                        if j % 6 == 0:
                            actual_score_lst.append(score_num)
                        elif j % 6 == 1:
                            gpt2_tuned_score_lst.append(score_num)
                        elif j % 6 == 2:
                            xlnet_score_lst.append(score_num)
                        elif j % 6 == 3:
                            lstm_attention_score_lst.append(score_num)
                        elif j % 6 == 4:
                            gpt2_untuned_bert_score_lst.append(score_num)
                        elif j % 6 == 5:
                            gpt2_untuned_score_lst.append(score_num)
                        j += 1
            line_count += 1
            
    response_count = line_count - 3    
    
    # Due to my stupidity, manual part3 - 20 has 119 repeated twice
    # The code below won't work for this scenario
    # input_dict3_nodup_lst = sorted(set(input_dict3_lst),key=input_dict3_lst.index)
    input_dict3_nodup_lst = []
    prev = -1
    for item in input_dict3_lst:
        if item != prev:
            input_dict3_nodup_lst.append(item)
        prev = item
    
    if verbose == 1:
        print(len(input_dict3_nodup_lst))
        print(len(actual_score_lst))
    
    df = pd.DataFrame(list(zip( 
            input_dict3_nodup_lst*response_count, 
            actual_score_lst, 
            gpt2_tuned_score_lst, 
            xlnet_score_lst, 
            lstm_attention_score_lst, 
            gpt2_untuned_bert_score_lst, 
            gpt2_untuned_score_lst 
            )), 
            columns =['Sim_index', 'Actual', 'gpt2_tuned', 'xlnet', 'lstm_attention', 'gpt2_untuned_bert', 'gpt2_untuned']) 
    
    if verbose == 1:
        print(df)
        
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    
    return df

In [56]:
df_all = None
for file_of_interest in survey_result_file_lst:
    df_of_interest = parse_survey_results(file_of_interest)
    print(df_of_interest.shape)
    if df_all is None:
        df_all = df_of_interest
    else:
        df_all = df_all.append(df_of_interest, ignore_index=True)
    
print(df_all.shape)

(70, 7)
(50, 7)
(30, 7)
(50, 7)
(200, 7)
